## Извлечение данных

In [10]:
import re
import pandas as pd
from datetime import datetime
import eval7, pprint
from poker.room.pokerstars import PokerStarsHandHistory
from poker.hand import Hand, Combo

In [12]:
data_pd = {'TypeOfRoom': [], 'HandId': [], 'Tournament': [], 'info': [], 'Level': [], 'Date': [],
           'Hour': [], 'TimeZoneAbb': [], 'Table': [], 'NoOfPlayers': [], 'bain': [], 'HandHiro': [], 'Board': []}

with open('dim100.txt', 'r') as f:
    data = f.read()

result = re.findall('(\w+) Hand #(\d*): Tournament #(\d*),(.*) Level (.*) - (\d{4}/\d+/\d+) (\d+\D\d+\D\d+) (\w*)', data)
Tables = re.findall('Table \'.*\' (.*) Seat .*\n', data)
bains = re.findall('(\D\d+\.\d+\+\D\d+\.\d+)', data)
HandHiros = re.findall("Dealt to dimetruk \D(.. ..)\D", data)
Boards = re.findall("Board \[(.*)\]", data)


s = re.findall('Seat (\d).*chips', data)
s = [int(x) for x in s]
seats = []
length = 1
for i in range(len(s)-1):
    if s[i+1] < s[i]:
        seats.append(length)
        length = 1
    else:
        length += 1
seats.append(length)


for r, table, bain, seat, handHiro, board in zip(result, Tables, bains, seats, HandHiros, Boards):
    data_pd['TypeOfRoom'].append(r[0])
    data_pd['HandId'].append(r[1])
    data_pd['Tournament'].append(r[2])
    data_pd['info'].append(r[3])
    data_pd['Level'].append(r[4])
    data_pd['Date'].append(r[5])
    data_pd['Hour'].append(r[6])
    data_pd['TimeZoneAbb'].append(r[7])
    data_pd['Table'].append(table) #table
    data_pd['NoOfPlayers'].append(seat)
    data_pd['bain'].append(bain)
    data_pd['HandHiro'].append(handHiro)
    data_pd['Board'].append(board)

df = pd.DataFrame(data_pd)

df.head()

,TypeOfRoom,HandId,Tournament,info,Level,Date,Hour,TimeZoneAbb,Table,NoOfPlayers,bain,HandHiro,Board
0,PokerStars,228182836411,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:46:28,MSK,9-max,3,$13.70+$1.30,7h 7s,5h 2h Kc 3h 5c
1,PokerStars,228182839161,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:46:45,MSK,9-max,3,$13.70+$1.30,Ac 5h,Jd Qh Qc 6d Kd
2,PokerStars,228182845929,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:47:30,MSK,9-max,3,$13.70+$1.30,5c 5s,Td Kh Th 2c Ad
3,PokerStars,228182848632,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:47:48,MSK,9-max,3,$13.70+$1.30,4c 4s,3h 6d 7d 8d 6h
4,PokerStars,228242890835,3228689682,$27.40+$2.60 USD Hold'em No Limit -,I (10/20),2021/07/12,21:25:36,MSK,9-max,9,$27.40+$2.60,9h 9s,As 4s Kc


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   TypeOfRoom   100 non-null    object
 1   HandId       100 non-null    object
 2   Tournament   100 non-null    object
 3   info         100 non-null    object
 4   Level        100 non-null    object
 5   Date         100 non-null    object
 6   Hour         100 non-null    object
 7   TimeZoneAbb  100 non-null    object
 8   Table        100 non-null    object
 9   NoOfPlayers  100 non-null    int64 
 10  bain         100 non-null    object
 11  HandHiro     100 non-null    object
 12  Board        100 non-null    object
dtypes: int64(1), object(12)
memory usage: 10.3+ KB


## Преобразуем строковые данные

In [13]:
before = type(df.Tournament[0])
print(before)

<class 'str'>


In [14]:
import numpy as np

# df.Tournament = df.Tournament.astype('int64')
df = df.astype({'HandId': np.int64, 'Tournament': np.int64})
df

,TypeOfRoom,HandId,Tournament,info,Level,Date,Hour,TimeZoneAbb,Table,NoOfPlayers,bain,HandHiro,Board
0,PokerStars,228182836411,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:46:28,MSK,9-max,3,$13.70+$1.30,7h 7s,5h 2h Kc 3h 5c
1,PokerStars,228182839161,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:46:45,MSK,9-max,3,$13.70+$1.30,Ac 5h,Jd Qh Qc 6d Kd
2,PokerStars,228182845929,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:47:30,MSK,9-max,3,$13.70+$1.30,5c 5s,Td Kh Th 2c Ad
3,PokerStars,228182848632,3227430197,$13.70+$1.30 USD Hold'em No Limit -,IX (200/400),2021/07/10,8:47:48,MSK,9-max,3,$13.70+$1.30,4c 4s,3h 6d 7d 8d 6h
4,PokerStars,228242890835,3228689682,$27.40+$2.60 USD Hold'em No Limit -,I (10/20),2021/07/12,21:25:36,MSK,9-max,9,$27.40+$2.60,9h 9s,As 4s Kc
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,PokerStars,228816365202,3242858326,$27.58+$2.42 USD Hold'em No Limit -,II (15/30),2021/08/05,1:12:39,MSK,6-max,6,$27.58+$2.42,5s 6s,3d 7d 2s 2h
96,PokerStars,228816445233,3242858326,$27.58+$2.42 USD Hold'em No Limit -,II (15/30),2021/08/05,1:16:14,MSK,6-max,5,$27.58+$2.42,8d 9c,5s Th 4c 7s Js
97,PokerStars,228816502665,3242858326,$27.58+$2.42 USD Hold'em No Limit -,II (15/30),2021/08/05,1:18:51,MSK,6-max,5,$27.58+$2.42,Td 6h,9c Kc Jc
98,PokerStars,228816568248,3242858326,$27.58+$2.42 USD Hold'em No Limit -,III (25/50),2021/08/05,1:21:53,MSK,6-max,5,$27.58+$2.42,Ts Kh,4c 9h 7c 5d 8s


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   TypeOfRoom   100 non-null    object
 1   HandId       100 non-null    int64 
 2   Tournament   100 non-null    int64 
 3   info         100 non-null    object
 4   Level        100 non-null    object
 5   Date         100 non-null    object
 6   Hour         100 non-null    object
 7   TimeZoneAbb  100 non-null    object
 8   Table        100 non-null    object
 9   NoOfPlayers  100 non-null    int64 
 10  bain         100 non-null    object
 11  HandHiro     100 non-null    object
 12  Board        100 non-null    object
dtypes: int64(3), object(10)
memory usage: 10.3+ KB


In [18]:
df[['HandHiro', 'Board', 'Date', 'Hour']].head()  # применим фильтер по столбцам

,HandHiro,Board,Date,Hour
0,7h 7s,5h 2h Kc 3h 5c,2021/07/10,8:46:28
1,Ac 5h,Jd Qh Qc 6d Kd,2021/07/10,8:46:45
2,5c 5s,Td Kh Th 2c Ad,2021/07/10,8:47:30
3,4c 4s,3h 6d 7d 8d 6h,2021/07/10,8:47:48
4,9h 9s,As 4s Kc,2021/07/12,21:25:36
